### plotting cc statistics and other misc issues

In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
print("strawberry")

# loss function and metrics
from trustworthai.utils.losses_and_metrics.dice_loss import DiceLossWithWeightedEmptySlices
from trustworthai.utils.losses_and_metrics.dice_loss_metric import DiceLossMetric, SsnDiceMeanMetricWrapper

# predefined training dataset
from trustworthai.utils.data_preprep.dataset_pipelines import load_data
from torch.utils.data import ConcatDataset

# fitter
from trustworthai.utils.fitting_and_inference.fitters.basic_lightning_fitter import StandardLitModelWrapper
from trustworthai.utils.fitting_and_inference.fitters.p_unet_fitter import PUNetLitModelWrapper
from trustworthai.utils.fitting_and_inference.get_trainer import get_trainer

# model
from trustworthai.journal_run.model_load.load_ssn import load_ssn
from trustworthai.journal_run.model_load.load_punet import load_p_unet
from trustworthai.journal_run.model_load.load_deterministic import load_deterministic
from trustworthai.journal_run.model_load.load_evidential import load_evidential
from trustworthai.models.stochastic_wrappers.ssn.LowRankMVCustom import LowRankMultivariateNormalCustom
from trustworthai.models.stochastic_wrappers.ssn.ReshapedDistribution import ReshapedDistribution

# optimizer and lr scheduler
import torch


import numpy as np
from tqdm import tqdm
import scipy.stats
from trustworthai.utils.plotting.saving_plots import save, imsave
from trustworthai.utils.print_and_write_func import print_and_write

# misc
import argparse
import os
import shutil
import shlex
from collections import defaultdict
from tqdm import tqdm
import sys
from natsort import natsorted

import pandas as pd
from trustworthai.analysis.connected_components.connected_comps_2d import conn_comp_2d_analysis
from trustworthai.analysis.evaluation_metrics.challenge_metrics import getAVD, getDSC, getHausdorff, getLesionDetection, do_challenge_metrics
from sklearn import metrics
import math

import torch
import matplotlib.pyplot as plt
from trustworthai.utils.plotting.saving_plots import save
from trustworthai.utils.print_and_write_func import print_and_write
from trustworthai.analysis.calibration.helper_funcs import *
from tqdm import tqdm
from trustworthai.utils.logits_to_preds import normalize_samples

# data
from trustworthai.utils.data_preprep.dataset_pipelines import load_clinscores_data, load_data, ClinScoreDataRetriever
from trustworthai.utils.uncertainty_maps.entropy_map import entropy_map_from_samples


# evaluation code
from trustworthai.journal_run.evaluation.new_scripts.eval_helper_functions import *
from trustworthai.journal_run.evaluation.new_scripts.model_predictions import *
from trustworthai.analysis.connected_components.connected_comps_2d import *
print("banana")


strawberry
banana


In [25]:
import SimpleITK as sitk

In [4]:
import matplotlib.pyplot as plt
import proplot as pplt
from trustworthai.utils.uncertainty_maps.entropy_map import entropy_map_from_samples

In [6]:
import seaborn as sns

In [8]:
from twaidata.torchdatasets_v2.individual_dataset_wrappers import *

In [9]:
VOXELS_TO_WMH_RATIO = 382
VOXELS_TO_WMH_RATIO_EXCLUDING_EMPTY_SLICES = 140

In [10]:
class ARGS():
    def __init__(self):
    
        # folder arguments
        self.ckpt_dir = '/home/s2208943/ipdis/results/journal_models/cross_validated_models/'
        self.model_name = "ssn_ens0_cv0"
        self.model_ckpts_folder = "/home/s2208943/ipdis/results/journal_models/cross_validated_models/" # "/home/s2208943/ipdis/results/cross_validated_models/"
        self.repo_dir = "/home/s2208943/ipdis/WMH_UQ_assessment"
        self.result_dir = "trustworthai/journal_run/interrater_experiments/results/"
        self.model_type = "ssn"
        self.uncertainty_type = "ssn"
        
        # data generation arguments
        self.dataset = 'chal'
        self.seed = 3407
        self.test_split = 0.15
        self.val_split = 0.15
        self.empty_slice_retention = 0.1
        
        # general arguments for the loss function
        self.loss_name = 'dice+xent'#'dicev2'#'dice+xent'
        self.dice_factor = 1#5
        self.xent_factor = 1#0.01
        self.xent_weight = 'none'
        self.xent_reweighting=1
        self.dice_empty_slice_weight = 0.5
        self.tversky_beta = 0.7
        self.reduction = 'mean'#'mean_sum'
        
        # evidential arguments
        self.kl_factor=0.1
        self.kl_anneal_count=452*4
        self.use_mle=0
        self.analytic_kl=0

        # p-unet arguments
        self.kl_beta=10.
        self.use_prior_for_dice="false"
        self.punet_sample_dice_coeff=0.05
        self.latent_dim=12

        # ssn arguments
        self.ssn_rank=25
        self.ssn_epsilon=1e-5
        self.ssn_mc_samples=10
        self.ssn_sample_dice_coeff=0.05
        self.ssn_pre_head_layers=32
        
        # training paradigm arguments
        self.lr = 2e-4
        self.dropout_p = 0.1
        self.encoder_dropout1 = 0
        self.encoder_dropout2 = 0
        self.decoder_dropout1 = 0
        self.decoder_dropout2 = 0
        self.max_epochs = 100
        self.early_stop_patience = 15
        self.batch_size = 8#32
        self.cross_validate = "true"
        self.cv_split = 0
        self.cv_test_fold_smooth = 1
        self.weight_decay = 0.0001
        self.overwrite = "true"
        self.no_test_fold = 'false'

In [11]:
args = ARGS()

# sanitise arguments
args.overwrite = True if args.overwrite.lower() == "true" else False
args.cross_validate = True if args.cross_validate.lower() == "true" else False
args.use_prior_for_dice = True if args.use_prior_for_dice.lower() == "true" else False
print(f"CHECKPOINT DIR: {args.ckpt_dir}")
#print(args)


# check if folder exists
model_result_folder = os.path.join(args.repo_dir, args.result_dir)
if not args.overwrite:
    existing_files = os.listdir(model_result_folder)
    for f in existing_files:
        if args.model_name + "_" in f:
            raise ValueError(f"ovewrite = false and model results exist! folder={model_result_folder}, model_name={args.model_name}")
with open(os.path.join(model_result_folder, f"{args.model_name}_init.txt"), "w") as f:
                      f.write("generating results\n")
        
# setup xent reweighting factor
XENT_VOXEL_RESCALE = VOXELS_TO_WMH_RATIO - (1-args.empty_slice_retention) * (VOXELS_TO_WMH_RATIO - VOXELS_TO_WMH_RATIO_EXCLUDING_EMPTY_SLICES)

XENT_WEIGHTING = XENT_VOXEL_RESCALE/2
args.xent_reweighting = XENT_WEIGHTING

CHECKPOINT DIR: /home/s2208943/ipdis/results/journal_models/cross_validated_models/


In [13]:
MODEL_LOADERS = {
    "deterministic":load_deterministic,
    "mc_drop":load_deterministic,
    "evidential":load_evidential,
    "ssn":load_ssn,
    "punet":load_p_unet,
}

MODEL_OUTPUT_GENERATORS = {
    "deterministic":mc_drop_mean_and_samples,
    "mc_drop":mc_drop_mean_and_samples,
    "evidential":evid_mean,
    "ssn":ssn_mean_and_samples,
    "punet":punet_mean_and_samples,
    "ind":ssn_mean_and_samples,
    "ens":ensemble_mean_and_samples,
    "ssn_ens":ssn_ensemble_mean_and_samples,
}

### setting up model and data

In [14]:
# get the 3d dataloader
ds_name = "mss3"
mss3_ds = MSS3InterRaterDataset()#WMHChallengeInterRaterDataset()#LBCInterRaterDataset()#MSS3InterRaterDataset()


In [15]:
len(mss3_ds)

68

In [16]:
xs, ys, ind, _ = mss3_ds[10]

In [17]:
xs.keys()

dict_keys(['FLAIR', 'T1', 'mask', 'vent_distance'])

In [18]:
ind

'MSS3_ED_003_V1'

In [19]:
xs.keys(), ys.keys()

(dict_keys(['FLAIR', 'T1', 'mask', 'vent_distance']),
 dict_keys(['lacune', 'wmhes', 'wmhmvh']))

In [20]:
xs['mask'].shape

torch.Size([53, 240, 240])

In [23]:
uncertainty_thresholds = torch.arange(0, 0.7, 0.01)

In [24]:
# load the model
model_dir = os.path.join(args.ckpt_dir, args.model_name)  
print("model dir: ", model_dir)
model_raw, loss, val_loss = MODEL_LOADERS[args.model_type](args)
model = load_best_checkpoint(model_raw, loss, model_dir, punet=args.model_type == "punet")

model dir:  /home/s2208943/ipdis/results/journal_models/cross_validated_models/ssn_ens0_cv0


In [26]:
xs3d_test = []
ys3d_test = []
inds_test = []
pv_region_masks = []

if ds_name == "mss3":
    r0_id = "wmhes"
    r1_id = "wmhmvh"
    ds_ir_folder = "MSS3_InterRaterData"
elif ds_name == "lbc":
    r0_id = "wmh"
    r1_id = "wmh_flthresh"
    ds_ir_folder = "LBC_InterRaterData"
elif ds_name == "challenge":
    r0_id = "wmho3"
    r1_id = "wmho4"
    ds_ir_folder = "WMHChallenge_InterRaterData"


for (xs, ys, ind, _) in tqdm(mss3_ds):
    if ind in ['MSS3_ED_073_V1', 'MSS3_ED_075_V1', 'MSS3_ED_078_V1', 'MSS3_ED_079_V1']:
        print("found")
        continue
    if r0_id in ys.keys() and r1_id in ys.keys():
        try:
            x = torch.stack([xs['FLAIR'], xs['T1'], xs['mask']], dim=0)
            y = torch.stack([ys[r0_id], ys[r1_id]], dim=0)

            vent_distance_map = xs['vent_distance']
        except:
            print(f"failed for {ind}")
            continue

        xs3d_test.append(x)
        ys3d_test.append(y)
        inds_test.append(ind)
        pv_region_masks.append((vent_distance_map < 10).type(torch.float32))

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                       | 60/68 [00:21<00:02,  3.24it/s]

found


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 61/68 [00:21<00:02,  3.23it/s]

found


 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏              | 63/68 [00:22<00:01,  3.14it/s]

found


 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 64/68 [00:22<00:01,  3.07it/s]

found


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:24<00:00,  2.82it/s]


In [27]:
rater0 = [y[0] for y in ys3d_test]
rater1 = [y[1] for y in ys3d_test]

In [28]:
rater_union = [y[0] * y[1] for y in ys3d_test]

In [29]:
rater0_ds = list(zip(xs3d_test, rater0))

In [30]:
from collections import defaultdict

In [31]:
gt_vols = [(torch.sum(y[0]).item(), torch.sum(y[1]).item()) for y in ys3d_test]

In [32]:
mean_gt_vols = [(torch.sum(y[0]).item() + torch.sum(y[1]).item())/2 for y in ys3d_test]

In [33]:
print(len(mean_gt_vols))

61


In [35]:
def soft_dice(pred, target):
    
    numerator = 2 * (pred * target).sum()
    denominator = (target**2).sum() + (pred**2).sum()
    
    return (numerator / denominator).item()


In [36]:
def fast_iou(pred, target):
    p1 = (pred == 1)
    t1 = (target == 1)
    intersection = (pred == 1) & (target == 1)
    numerator = intersection.sum()
    denominator = p1.sum() + t1.sum() - numerator
    return (numerator/(denominator + 1e-30)).item()

In [37]:
raters = [rater0, rater1]
rater_results = [defaultdict(lambda : {}) for _ in range(len(raters))]

num_samples_list = [2, 3, 5, 7, 10, 15, 20, 25, 30]
num_samples = 10
print("NUM SAMPLES: ", num_samples)
args.eval_sample_num = num_samples

# load the predictions
print("GENERATING PREDICTIONS")
print(args.uncertainty_type)
means, samples, misc = get_means_and_samples(model_raw, rater0_ds, num_samples=num_samples, model_func=MODEL_OUTPUT_GENERATORS[args.uncertainty_type], args=args)

uncertainty_thresholds = torch.arange(0, 0.7, 0.01)
ent_maps = get_uncertainty_maps(means, samples, misc, args)

NUM SAMPLES:  10
GENERATING PREDICTIONS
ssn


  0%|                                                                                                                                                                                                                  | 0/61 [00:00<?, ?it/s]/home/s2208943/miniconda3/envs/wmh/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:30<00:00,  2.03it/s]


ssn
generating uncertainty maps


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:06<00:00,  9.94it/s]


In [38]:
len(rater0_ds)

61

In [39]:
len(samples)

61

In [42]:
import cc3d

In [43]:
def find_non_overlapping_ccs(ccs_y0, y1):
    no_overlap_ccs = []
    ccs_size = []
    no_overlap_ccs_img = torch.zeros(y1.shape, device=y1.device, dtype=y1.dtype)
    
    for cc_id in ccs_y0.unique():
        if cc_id == 0:
            continue
        cc = ccs_y0 == cc_id
        if (cc * y1).sum() == 0:
            no_overlap_ccs.append(int(cc_id.item()))
            ccs_size.append(cc.sum().item())
            no_overlap_ccs_img += cc
    
    return no_overlap_ccs, ccs_size, no_overlap_ccs_img

In [44]:
def remove_ccids_with_0IOU(ccs_y0, ccs_y1, y0, y1):
    y0_ccids = ccs_y0.unique().type(torch.int32).tolist()
    y1_ccids = ccs_y1.unique().type(torch.int32).tolist()
    y0_ccids.remove(0)
    y1_ccids.remove(0)
    zero_overlap_y0 = []
    zero_overlap_y1 = []
    for cc_id in y0_ccids:
        cc = ccs_y0 == cc_id
        if (cc * y1).sum() == 0:
            zero_overlap_y0.append(cc_id)
    for cc_id in y1_ccids:
        if cc_id == 0:
            continue
        cc = ccs_y1 == cc_id
        if (cc * y0).sum() == 0:
            zero_overlap_y1.append(cc_id)
    for cc_id in zero_overlap_y0:
        y0_ccids.remove(cc_id)
    for cc_id in zero_overlap_y1:
        y1_ccids.remove(cc_id)
        
    return y0_ccids, y1_ccids

def find_ious_between_raters(ccs_y0, ccs_y1, y0, y1):
    
    # first narrow down the list to all the connected components that do not have zero overlap
    y0_ccids, y1_ccids = remove_ccids_with_0IOU(ccs_y0, ccs_y1, y0, y1)
    
    # loop through ccs to find the one with the highest iou
    cc_ious = []
    ccs_y1_match = []
    sizes = []
    match_sizes = []
    for cc_id0 in y0_ccids:
        cc0 = ccs_y0 == cc_id0
        cc0_sum = cc0.sum().item()
        best_iou = 0
        best_iou_cc_id = None
        match_size = None
        for cc_id1 in y1_ccids:
            cc1 = ccs_y1 == cc_id1
            iou = (cc0 & cc1).sum() / ((cc1 + cc0) > 0).sum()
            iou = iou.item()
            if iou >= best_iou:
                best_iou = iou
                best_iou_cc_id = cc_id1
                match_size = cc1.sum().item()
        # print(best_iou_cc_id)
        # y1_ccids.remove(best_iou_cc_id)
        cc_ious.append(best_iou)
        ccs_y1_match.append(best_iou_cc_id)
        sizes.append(cc0_sum)
        match_sizes.append(match_size)
        
    return cc_ious, y0_ccids, ccs_y1_match, sizes, match_sizes
            

In [45]:
def create_overlap_image(y0_cc_ids, y1_cc_ids, y0_ccs, y1_ccs):
    overlap_ccs_img = torch.zeros(y1_ccs.shape, device=y1_ccs.device, dtype=y1_ccs.dtype)
    for cc_id in y0_cc_ids:
        cc = y0_ccs == cc_id
        overlap_ccs_img[cc] = 1
    if y1_cc_ids is not None:
        for cc_id in y1_cc_ids:
            cc = y1_ccs == cc_id
            overlap_ccs_img[cc] = 1
        
    return overlap_ccs_img

In [46]:
from scipy.ndimage import distance_transform_edt
import SimpleITK as sitk
from twaidata.MRI_preprep.resample import get_resampled_img

def get_distance_map(img, rescale_ratio=3):
    
    # resample to 1x1x1 space
    device = img.device
    img = img.type(torch.float32)
    img = torch.nn.functional.interpolate(img.unsqueeze(0).unsqueeze(0), size=None, scale_factor=(rescale_ratio, 1, 1), mode='nearest', align_corners=None, recompute_scale_factor=None, antialias=False)
    img = img.squeeze().cpu().numpy()
    
    # compute distance map
    distance_map = distance_transform_edt(1 - img)
    
    # resample back to original space
    distance_map = torch.from_numpy(distance_map).to(device)
    distance_map = torch.nn.functional.interpolate(distance_map.unsqueeze(0).unsqueeze(0), size=None, scale_factor=(1/rescale_ratio, 1, 1), mode='trilinear', align_corners=None, recompute_scale_factor=None, antialias=False)
    distance_map = distance_map.squeeze()
    
    return distance_map


# ### UIRO per region analysis.

# no_overlap_distance_map = get_distance_map(combined_no_overlap_image)
# low_overlap_distance_map = get_distance_map(combined_low_overlap_diff_image)
# high_overlap_distance_map = get_distance_map(combined_high_overlap_diff_image)

### split it into pv and deep


## do whole image UIRO and JUEO in pv region and deep region. that can go along with the other analysis. Nice.

In [47]:
def IOU_region_UIRO(ent, rater_diff, no_overlap_distance_map, low_overlap_distance_map, high_overlap_distance_map):
        uiro_no_overlap = []
        uiro_low_overlap = []
        uiro_high_overlap = []
        
        d = 5
        no_exclusion_region = (no_overlap_distance_map < d) * (low_overlap_distance_map > d) * (high_overlap_distance_map > d)
        low_exclusion_region = (no_overlap_distance_map > d) * (low_overlap_distance_map < d) * (high_overlap_distance_map > d)
        high_exclusion_region = (no_overlap_distance_map > d) * (low_overlap_distance_map > d) * (high_overlap_distance_map < d)
        
        for t in uncertainty_thresholds:
            e = ent > t
            uiro_no_overlap.append(fast_dice(e * no_exclusion_region, rater_diff * no_exclusion_region))
            uiro_low_overlap.append(fast_dice(e * low_exclusion_region, rater_diff * low_exclusion_region))
            uiro_high_overlap.append(fast_dice(e * high_exclusion_region, rater_diff * high_exclusion_region))
        
        
        return uiro_no_overlap, uiro_low_overlap, uiro_high_overlap

## run cc analysis across the whole image

In [ ]:
all_sizes = []
all_ious = []
all_match_sizes = []
IOU0_ccs_sizes = []

ent_no_overlap_all, ent_low_overlap_all, ent_high_overlap_all, ent_exact_overlap_all = [], [], [], []

none_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
low_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
high_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
exact_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
non_useful_ccs_all = [[] for _ in range(len(uncertainty_thresholds))]

num_FP_uncertainty_ccs_all = []
mean_size_FP_uncertainty_ccs_all = []

none_cc_mean = []
low_cc_mean = []
high_cc_mean = []
exact_cc_mean = []

uiro_no_overlap_all = []
uiro_low_overlap_all = []
uiro_high_overlap_all = []

for i in tqdm(range(len(means))):
    x = xs3d_test[i].cuda()
    pred = means[i].cuda().argmax(dim=1)
    e = ent_maps[i].cuda()
    y0 = rater0[i]
    y1 = rater1[i]
    
    ccs_y0 = cc3d.connected_components(y0.type(torch.int32).numpy(), connectivity=26) # 26-connected
    ccs_y0 = torch.from_numpy(ccs_y0.astype(np.float32)).cuda()
    
    ccs_y1 = cc3d.connected_components(y1.type(torch.int32).numpy(), connectivity=26) # 26-connected
    ccs_y1 = torch.from_numpy(ccs_y1.astype(np.float32)).cuda()
    
    y0 = y0.cuda()
    y1 = y1.cuda()
    
    diff_image = (y0 + y1) == 1
    any_prediction_image = (pred + y0 + y1) > 0 # anywhere where either the model or the raters predict
    
    
    # find type 1: connected components in either map with no overlap
    no_overlap_ccs_y0, no_overlap_ccs_size_y0, no_overlap_ccs_img_y0 = find_non_overlapping_ccs(ccs_y0, y1)
    no_overlap_ccs_y1, no_overlap_ccs_size_y1, no_overlap_ccs_img_y1 = find_non_overlapping_ccs(ccs_y1, y0)
    IOU0_ccs_sizes.extend(no_overlap_ccs_size_y0 + no_overlap_ccs_size_y1)
    combined_no_overlap_image = (no_overlap_ccs_img_y0 + no_overlap_ccs_img_y1).type(torch.int)
    
    # get information on the IOU of connected components between rater 0 and rater 1 where the IOU > 0
    ious, y0_ccids, y1_ccid_matches, sizes, match_sizes = find_ious_between_raters(ccs_y0, ccs_y1, y0, y1)
    all_ious.append(ious)
    all_sizes.append(sizes)
    all_match_sizes.append(match_sizes)
    
    ious = torch.Tensor(ious)
    y0_ccids = torch.Tensor(y0_ccids)
    y1_ccid_matches = torch.Tensor(y1_ccid_matches)
    
    # find type 2: areas where connected components have a poor overlap IOU of 0.5 and below but not 0.
    # IOU of zero ccs have already been discounted in the previous step.
    low_ious = ious < 0.5
    low_IOU_y0_ccs = y0_ccids[low_ious]
    low_IOU_y1_ccs = y1_ccid_matches[low_ious]
    low_IOU_y1_ccs = torch.unique(low_IOU_y1_ccs)
    combined_low_overlap_image = create_overlap_image(low_IOU_y0_ccs, low_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_low_overlap_diff_image = combined_low_overlap_image * diff_image
    
    # find type 3: high overlap
    high_ious = (ious >= 0.5) * (ious < 1)
    high_IOU_y0_ccs = y0_ccids[high_ious]
    high_IOU_y1_ccs = y1_ccid_matches[high_ious]
    high_IOU_y1_ccs = torch.unique(high_IOU_y1_ccs)
    combined_high_overlap_image = create_overlap_image(high_IOU_y0_ccs, high_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_high_overlap_diff_image = combined_high_overlap_image * diff_image
    
    # find type 4: areas of exact overlap
    exact_ious = ious == 1
    exact_IOU_y0_ccs = y0_ccids[exact_ious]
    exact_IOU_y1_ccs = y1_ccid_matches[exact_ious]
    exact_IOU_y1_ccs = torch.unique(exact_IOU_y1_ccs)
    combined_exact_overlap_image = create_overlap_image(exact_IOU_y0_ccs, None, ccs_y0, ccs_y1).type(torch.int)
    
    # then get the areas where the raters agree and remove those areas. see what we are left with...
    # that is the joint uncertainty error overlap... but I want to look at the overlap away from the boundaries....
    
    # my concern is that we still have all of this edge information.
    
    # ANALYSIS 1 cc average uncertainty information
    # we need average uncertainty per connected component in the following regions
    # - IR IOU 0
    # - IR IOU 1
    # - diff of IR 0 < IOU < 0.5 regions
    # - diff of IR 0.5 <= IOU < 1 regions
    
    # ANALYSIS 1.5 pixel wise information
    # the same as analysis 1 but with individual voxels
    # ef = e.flatten()
    # ent_no_overlap, ent_low_overlap, ent_high_overlap, ent_exact_overlap = ef[combined_no_overlap_image.flatten()==1], ef[combined_low_overlap_diff_image.flatten()==1], ef[combined_high_overlap_diff_image.flatten()==1], ef[combined_exact_overlap_image.flatten()==1]
    # ent_no_overlap_all.extend(ent_no_overlap.cpu())
    # ent_low_overlap_all.extend(ent_low_overlap.cpu())
    # ent_high_overlap_all.extend(ent_high_overlap.cpu())
    # ent_exact_overlap_all.extend(ent_exact_overlap.cpu())
    
    
    # Analysis 2 connected component extra analysis
    # as the uncertainty threshold increases we look at:
    # average proportion of IR IOU 0 that are uncertain
    # average proportion of IR low IOU that are uncertain
    # average proportion of IR IOU high that are uncertain
    # average proportion of IR IOU 1 that are uncertain
    # number of connected components in uncertainty map that have zero overlap with either rater. I should do this for a fewer number of connected components
    # none_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    # low_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    # high_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    # exact_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    # non_useful_ccs = [[] for _ in range(len(uncertainty_thresholds))]
    # num_FP_uncertainty_ccs = []
    # mean_size_FP_uncertainty_ccs = []
    
    ccs_none = torch.from_numpy(cc3d.connected_components(combined_no_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_low = torch.from_numpy(cc3d.connected_components(combined_low_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_high = torch.from_numpy(cc3d.connected_components(combined_high_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_exact = torch.from_numpy(cc3d.connected_components(combined_exact_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    
#     any_prediction_image_dist_map = get_distance_map(any_prediction_image)
#     any_prediction_image_dist_map = any_prediction_image_dist_map > 5 # 
    
    # print(ccs_none.unique())
    # print(combined_no_overlap_image.sum())
    for ti, t in enumerate(uncertainty_thresholds):
        et = (e > t).type(torch.float32)
        
        # look at how many ccs there are
        # non_overlapping_uncert_region = et * any_prediction_image_dist_map
        # non_overlapping_uncert_ccs = torch.from_numpy(cc3d.connected_components(non_overlapping_uncert_region.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
        # num_FP_uncertainty_ccs.append(len(non_overlapping_uncert_ccs.unique()) - 1) # -1 to get rid of background class
        # mean_size_FP_uncertainty_ccs.append(
        #     torch.Tensor(
        #         [(non_overlapping_uncert_ccs == cc).sum().item() for cc in non_overlapping_uncert_ccs.unique() if cc != 0]
        #     ).mean()
        # )
        
        # find proportion uncertain of each cc in each region
        for cc_id in ccs_none.unique():
            if cc_id == 0:
                continue
            cc = ccs_none == cc_id
            # print(cc_id)
            # print(cc.sum())
            none_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                none_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_low.unique():
            if cc_id == 0:
                continue
            cc = ccs_low == cc_id
            low_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                low_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_high.unique():
            if cc_id == 0:
                continue
            cc = ccs_high == cc_id
            high_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                high_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_exact.unique():
            if cc_id == 0:
                continue
            cc = ccs_exact == cc_id
            exact_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                exact_cc_mean.append(e[cc==1].mean())
                
#     num_FP_uncertainty_ccs_all.append(num_FP_uncertainty_ccs)
#     mean_size_FP_uncertainty_ccs_all.append(mean_size_FP_uncertainty_ccs)
                
    # ### UIRO per region analysis.
    # no_overlap_distance_map = get_distance_map(combined_no_overlap_image)
    # low_overlap_distance_map = get_distance_map(combined_low_overlap_diff_image)
    # high_overlap_distance_map = get_distance_map(combined_high_overlap_diff_image)
    # uiro_no_overlap, uiro_low_overlap, uiro_high_overlap = IOU_region_UIRO(e, diff_image, no_overlap_distance_map, low_overlap_distance_map, high_overlap_distance_map)
    # uiro_no_overlap_all.append(uiro_no_overlap)
    # uiro_low_overlap_all.append(uiro_low_overlap)
    # uiro_high_overlap_all.append(uiro_high_overlap)
    
    # break
                
    

 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                    | 50/61 [03:31<00:42,  3.86s/it]

In [ ]:
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=3, ncols=8)

fontsize = 16

if ds_name == "challenge":
    sel_slices = [22, 24, 27]
    
elif ds_name == "lbc":
    sel_slices = [29, 31, 35]

elif ds_name == "mss3":
    sel_slices = [22, 27, 33]

for row, islice in enumerate(sel_slices):
    PV_region = pv_region_masks[i]
    PV_region_img = torch.ones([*PV_region.shape, 4])

    PV_region_img[:,:,:,2] = 0.5 # create a pastel yellow
    PV_region_img[:,:,:,3] = PV_region # set the rest of the voxels to have alpha of zero.

    
    b = 20
    axs[0 + row*8].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray', vmin=-3, vmax=2.5)
    axs[0 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[0 + row*8].set_title("FLAIR", fontsize=fontsize)
    axs[1 + row*8].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[1 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[1 + row*8].set_title("analyst 1", fontsize=fontsize)
    axs[2 + row*8].imshow(y1.cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[2 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[2 + row*8].set_title("analyst 2 ", fontsize=fontsize)
    axs[3 + row*8].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[3 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[3 + row*8].set_title("ccs with no overlap  \n(in 3D) JI = 0", fontsize=fontsize)
    axs[4 + row*8].imshow(combined_low_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[4 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[4 + row*8].set_title("ccs with low overlap \n(in 3D) 0 < JI < 0.5 ", fontsize=fontsize)
    axs[5 + row*8].imshow(combined_high_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[5 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[5 + row*8].set_title("ccs with high overlap  \n(in 3D) 0.5 <= JI < 1", fontsize=fontsize)
    axs[6 + row*8].imshow(combined_exact_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
    axs[6 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[6 + row*8].set_title("ccs with exact overlap \n(in 3D) JI = 1", fontsize=fontsize)
    axs[7 + row*8].imshow(ent_maps[i].cpu()[islice][b:-b, b:-b], cmap='magma')
    axs[7 + row*8].imshow(PV_region_img[islice][b:-b, b:-b], alpha=0.5)
    if row == 0:
        axs[7 + row*8].set_title("Uncertainty", fontsize=fontsize)
    axs.format(grid=False, xlocator='null', ylocator='null')
    for ax in axs:
        for spine in ax.spines:
            ax.spines[spine].set_color('none')
        
fig.savefig(f"Example of overlap types under consideration {ds_name}.png", bbox_inches='tight', dpi=600)

In [ ]:
islice = 30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=7)
b = 20
axs[0].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[1].set_title("rater 0")
axs[2].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[2].set_title("ccs with no overlap (in 3D) \n IOU = 0")
axs[3].imshow((diff_image * combined_low_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[3].set_title("low IOU disagreement region")
axs[4].imshow((diff_image * combined_high_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')

axs[4].set_title("high IOU disagreement region")
axs[5].imshow(ent_maps[i].cpu()[islice][b:-b, b:-b], cmap='Oranges')
axs[5].set_title("uncertainty")
axs[6].imshow(means[i][islice].argmax(dim=0).cpu()[b:-b, b:-b], cmap='Greens')
axs[6].set_title("model prediction")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')
        
# fig.savefig("Example of overlap and uncertainty.png", bbox_inches='tight', dpi=600)

In [ ]:
islice=30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=5)
axs[0].imshow(x.cpu()[0, islice], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.cpu()[islice], cmap='gray')
axs[1].set_title("rater 0")
axs[2].imshow(ccs_y0.cpu()[islice], cmap='Set2')
axs[2].set_title("rater 0 ccs")
axs[3].imshow(y1.cpu()[islice], cmap='gray')
axs[3].set_title("rater 1 ")
axs[4].imshow(ccs_y1.cpu()[islice], cmap='Set2')
axs[4].set_title("rater 1 ccs")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')

In [ ]:
all_ious = torch.cat([torch.Tensor(t) for t in all_ious])
all_sizes = torch.cat([torch.Tensor(t) for t in all_sizes])
all_match_sizes = torch.cat([torch.Tensor(t) for t in all_match_sizes])
IOU0_ccs_sizes = torch.Tensor(IOU0_ccs_sizes)

In [ ]:
import seaborn as sns
from matplotlib.ticker import NullLocator

In [ ]:
fig = pplt.figure(space=6, refwidth='20em', share=False)
axs = fig.subplots(nrows=1, ncols=4)

scale = 0.003
scaled_all_sizes = all_sizes * scale
scaled_IOU0_ccs_sizes = IOU0_ccs_sizes * scale
scaled_all_match_sizes = all_match_sizes * scale

axs[0].hist2d(all_ious[(scaled_all_sizes< 0.3) * (all_ious<1)].tolist(), (scaled_all_sizes[(scaled_all_sizes< 0.3) * (all_ious<1)]).tolist(), bins=10, colorbar='b',
    colorbar_kw={'length': 1, 'label': "Count"})
axs[0].set_xlabel("Conn. Comp. JI between analysts \n (only when 0 < JI < 1)")
axs[0].set_ylabel(r"Conn. Comp. Volume (ml)")
axs[0].set_title("Analyst JI vs Voxel count histogram per \n Connected Component " + r"(Vol. < 0.3 ml)")

axs[1].scatter((scaled_all_sizes[all_ious<1]).tolist(), (scaled_all_match_sizes[all_ious<1]).tolist(), c=all_ious[all_ious<1].tolist(), alpha=0.5, cmap='summer', colorbar='b',
    colorbar_kw={'length': 1, 'label': "JI"})
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel(r"Analyst 1 Conn. Comp. Volume (ml)")
axs[1].set_ylabel(r"Analyst 2 Conn. Comp. Volume (ml)")
axs[1].set_title("Volume of corresponding Connected \n Components between analysts (0 < JI < 1)")
axs[1].set_xlim((None, 100))
axs[1].set_ylim((None, 100))

axs[2].hist(all_ious.tolist(), bins=20, color='goldenrod')
axs[2].set_xlabel("Conn. Comp. JI between analysts")
axs[2].set_ylabel("Count")
axs[2].set_title("Histogram of Connected Component JI \n between analysts (JI > 0)")

ji0_arr = scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes < 0.3].tolist()
bins = len(ji0_arr) // 30
axs[3].hist(ji0_arr, bins=bins, label='JI = 0', color='#d50032')
ji1_arr = (scaled_all_sizes[(all_ious==1) * (scaled_all_sizes < 0.3)]).tolist()
bins = len(ji1_arr) // 30
if bins > 0:
    axs[3].hist(ji1_arr, bins=bins, label='JI = 1', color='#002c53')
axs[3].legend(loc='b')
axs[3].set_xlabel(r"Conn. Comp. Volume (ml)")
axs[3].set_ylabel("Count")
axs[3].set_title("Histogram of Volume of Conn. Comps. \n with JI of 0 or 1 between analysts")
axs[3].set_xlim((0, 0.3))

axs.format(abc=True, abcloc='ul')
for ax in axs:
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
    ax.spines['left'].set_linewidth(1.2)  # Thicker left spine

fig.save(f"Connected Component information {ds_name}.pdf", bbox_inches="tight")

In [ ]:
len((scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes < 0.3]).tolist()) / 20

In [ ]:
plt.hist([scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes > 0.3].tolist(), scaled_all_sizes[(all_ious==1) * (scaled_all_sizes > 0.3)].tolist()], color=['#d50032', '#002c53'], bins=20, label=['JI = 0', 'JI = 1'])
plt.legend()
plt.xlabel(r"Conn. Comp. Volume (ml)")
plt.ylabel("Count")
plt.title("Histogram of sizes of large Conn. Comps. \n with JI of 0 or 1 between analysts")

ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
ax.spines['left'].set_linewidth(1.2)  # Thicker left spine
    
plt.savefig(f"Size of large CCS with 0or1 JI {ds_name}.pdf", bbox_inches="tight")



## run cc analysis across PV region

In [ ]:
# for i in range(len(means)):
#     print(xs3d_test[i].shape, pv_region_masks[i].shape)

In [ ]:
all_sizes = []
all_ious = []
all_match_sizes = []
IOU0_ccs_sizes = []

ent_no_overlap_all, ent_low_overlap_all, ent_high_overlap_all, ent_exact_overlap_all = [], [], [], []

none_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
low_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
high_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
exact_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
non_useful_ccs_all = [[] for _ in range(len(uncertainty_thresholds))]

none_cc_mean = []
low_cc_mean = []
high_cc_mean = []
exact_cc_mean = []

for i in tqdm(range(len(means))):
    x = xs3d_test[i].cuda()
    pred = means[i].cuda().argmax(dim=1)
    e = ent_maps[i].cuda()
    pv_region = pv_region_masks[i].cuda()
    if x.shape[1] != pv_region.shape[0] or x.shape[2] != pv_region.shape[1] or x.shape[3] != pv_region.shape[2]:
        continue
    y0 = rater0[i].cuda() * pv_region
    y1 = rater1[i].cuda() * pv_region
    
    ccs_y0 = cc3d.connected_components(y0.type(torch.int32).cpu().numpy(), connectivity=26) # 26-connected
    ccs_y0 = torch.from_numpy(ccs_y0.astype(np.float32)).cuda()
    
    ccs_y1 = cc3d.connected_components(y1.type(torch.int32).cpu().numpy(), connectivity=26) # 26-connected
    ccs_y1 = torch.from_numpy(ccs_y1.astype(np.float32)).cuda()
    
    y0 = y0.cuda()
    y1 = y1.cuda()
    
    diff_image = (y0 + y1) == 1
    
    # find type 1: connected components in either map with no overlap
    no_overlap_ccs_y0, no_overlap_ccs_size_y0, no_overlap_ccs_img_y0 = find_non_overlapping_ccs(ccs_y0, y1)
    no_overlap_ccs_y1, no_overlap_ccs_size_y1, no_overlap_ccs_img_y1 = find_non_overlapping_ccs(ccs_y1, y0)
    IOU0_ccs_sizes.extend(no_overlap_ccs_size_y0 + no_overlap_ccs_size_y1)
    combined_no_overlap_image = (no_overlap_ccs_img_y0 + no_overlap_ccs_img_y1).type(torch.int)
    
    # get information on the IOU of connected components between rater 0 and rater 1 where the IOU > 0
    ious, y0_ccids, y1_ccid_matches, sizes, match_sizes = find_ious_between_raters(ccs_y0, ccs_y1, y0, y1)
    all_ious.append(ious)
    all_sizes.append(sizes)
    all_match_sizes.append(match_sizes)
    
    ious = torch.Tensor(ious)
    y0_ccids = torch.Tensor(y0_ccids)
    y1_ccid_matches = torch.Tensor(y1_ccid_matches)
    
    # find type 2: areas where connected components have a poor overlap IOU of 0.5 and below but not 0.
    # IOU of zero ccs have already been discounted in the previous step.
    low_ious = ious < 0.5
    low_IOU_y0_ccs = y0_ccids[low_ious]
    low_IOU_y1_ccs = y1_ccid_matches[low_ious]
    low_IOU_y1_ccs = torch.unique(low_IOU_y1_ccs)
    combined_low_overlap_image = create_overlap_image(low_IOU_y0_ccs, low_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_low_overlap_diff_image = combined_low_overlap_image * diff_image
    
    # find type 3: high overlap
    high_ious = (ious >= 0.5) * (ious < 1)
    high_IOU_y0_ccs = y0_ccids[high_ious]
    high_IOU_y1_ccs = y1_ccid_matches[high_ious]
    high_IOU_y1_ccs = torch.unique(high_IOU_y1_ccs)
    combined_high_overlap_image = create_overlap_image(high_IOU_y0_ccs, high_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_high_overlap_diff_image = combined_high_overlap_image * diff_image
    
    # find type 4: areas of exact overlap
    exact_ious = ious == 1
    exact_IOU_y0_ccs = y0_ccids[exact_ious]
    exact_IOU_y1_ccs = y1_ccid_matches[exact_ious]
    exact_IOU_y1_ccs = torch.unique(exact_IOU_y1_ccs)
    combined_exact_overlap_image = create_overlap_image(exact_IOU_y0_ccs, None, ccs_y0, ccs_y1).type(torch.int)
    
    # then get the areas where the raters agree and remove those areas. see what we are left with...
    # that is the joint uncertainty error overlap... but I want to look at the overlap away from the boundaries....
    
    # my concern is that we still have all of this edge information.
    
    # ANALYSIS 1 cc average uncertainty information
    # we need average uncertainty per connected component in the following regions
    # - IR IOU 0
    # - IR IOU 1
    # - diff of IR 0 < IOU < 0.5 regions
    # - diff of IR 0.5 <= IOU < 1 regions
    
    # ANALYSIS 1.5 pixel wise information
    # the same as analysis 1 but with individual voxels
    ef = e.flatten()
    ent_no_overlap, ent_low_overlap, ent_high_overlap, ent_exact_overlap = ef[combined_no_overlap_image.flatten()==1], ef[combined_low_overlap_diff_image.flatten()==1], ef[combined_high_overlap_diff_image.flatten()==1], ef[combined_exact_overlap_image.flatten()==1]
    ent_no_overlap_all.extend(ent_no_overlap.cpu())
    ent_low_overlap_all.extend(ent_low_overlap.cpu())
    ent_high_overlap_all.extend(ent_high_overlap.cpu())
    ent_exact_overlap_all.extend(ent_exact_overlap.cpu())
    
    
    # Analysis 2 connected component extra analysis
    # as the uncertainty threshold increases we look at:
    # average proportion of IR IOU 0 that are uncertain
    # average proportion of IR low IOU that are uncertain
    # average proportion of IR IOU high that are uncertain
    # average proportion of IR IOU 1 that are uncertain
    # number of connected components in uncertainty map that have zero overlap with either rater. I should do this for a fewer number of connected components
    none_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    low_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    high_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    exact_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    non_useful_ccs = [[] for _ in range(len(uncertainty_thresholds))]
    
    ccs_none = torch.from_numpy(cc3d.connected_components(combined_no_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_low = torch.from_numpy(cc3d.connected_components(combined_low_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_high = torch.from_numpy(cc3d.connected_components(combined_high_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_exact = torch.from_numpy(cc3d.connected_components(combined_exact_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    # print(ccs_none.unique())
    # print(combined_no_overlap_image.sum())
    for ti, t in enumerate(uncertainty_thresholds):
        et = (e > t).type(torch.float32)
        
        for cc_id in ccs_none.unique():
            if cc_id == 0:
                continue
            cc = ccs_none == cc_id
            # print(cc_id)
            # print(cc.sum())
            none_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                none_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_low.unique():
            if cc_id == 0:
                continue
            cc = ccs_low == cc_id
            low_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                low_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_high.unique():
            if cc_id == 0:
                continue
            cc = ccs_high == cc_id
            high_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                high_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_exact.unique():
            if cc_id == 0:
                continue
            cc = ccs_exact == cc_id
            exact_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                exact_cc_mean.append(e[cc==1].mean())
                
                
    # Analysis 2.5:
    # average number of superfluous connected components across the dataset as the threshold increases.
            
    
    # Analysis 3 UIRO outside of the regions of the segmentation (do euclidian distance or different components, square and cross for example?) Nice.
    
    # Analysis 4 JUEO outside the regions of the segmentation produced by the model (provide this with 2 different components or do euclidian distance?)
    
    # Analysis 5 pcitures
    # Collect a number of images of different individuals, the different maps, as I have outlined below, and demonstrate the things that we are talking about.
    # Just provide an image every 10 patients or something to demonstrate the examples. 
    # Nice.
    
    # yes this is a really good way to approach the analysis and I actually like it quite a lot.
    # then I need to collect my nice plot of connected components for each dataset and also collect some nice visual examples from the different datasets as well.....
    
    
    # if i == 1:
    #     break
    # break

In [ ]:
islice=30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=7)
b = 20
axs[0].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(y1.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[2].set_title("analyst 2 ")
axs[3].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[3].set_title("ccs with no overlap (in 3D) \n IOU = 0")
axs[4].imshow(combined_low_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[4].set_title("ccs with low overlap (in 3D)\n 0 < IOU < 0.5 ")
axs[5].imshow(combined_high_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[5].set_title("ccs with high overlap (in 3D) \n 0.5 <= IOU < 1")
axs[6].imshow(combined_exact_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[6].set_title("ccs with exact overlap (in 3D)\n IOU = 1")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')
        
fig.savefig("Example of overlap types under consideration.png", bbox_inches='tight', dpi=600)

In [ ]:
islice = 30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=7)
b = 20
axs[0].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[2].set_title("ccs with no overlap (in 3D) \n IOU = 0")
axs[3].imshow((diff_image * combined_low_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[3].set_title("low IOU disagreement region")
axs[4].imshow((diff_image * combined_high_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')

axs[4].set_title("high IOU disagreement region")
axs[5].imshow(ent_maps[i].cpu()[islice][b:-b, b:-b], cmap='Oranges')
axs[5].set_title("uncertainty")
axs[6].imshow(means[i][islice].argmax(dim=0).cpu()[b:-b, b:-b], cmap='Greens')
axs[6].set_title("model prediction")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')
        
# fig.savefig("Example of overlap and uncertainty.png", bbox_inches='tight', dpi=600)

In [ ]:
islice=30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=5)
axs[0].imshow(x.cpu()[0, islice], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.cpu()[islice], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(ccs_y0.cpu()[islice], cmap='Set2')
axs[2].set_title("analyst 1 ccs")
axs[3].imshow(y1.cpu()[islice], cmap='gray')
axs[3].set_title("analyst 2 ")
axs[4].imshow(ccs_y1.cpu()[islice], cmap='Set2')
axs[4].set_title("analyst 2 ccs")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')

In [ ]:
all_ious = torch.cat([torch.Tensor(t) for t in all_ious])
all_sizes = torch.cat([torch.Tensor(t) for t in all_sizes])
all_match_sizes = torch.cat([torch.Tensor(t) for t in all_match_sizes])
IOU0_ccs_sizes = torch.Tensor(IOU0_ccs_sizes)

In [ ]:
import seaborn as sns
from matplotlib.ticker import NullLocator

In [ ]:
fig = pplt.figure(space=6, refwidth='20em', share=False)
axs = fig.subplots(nrows=1, ncols=4)

scale = 0.003
scaled_all_sizes = all_sizes * scale
scaled_IOU0_ccs_sizes = IOU0_ccs_sizes * scale
scaled_all_match_sizes = all_match_sizes * scale

axs[0].hist2d(all_ious[(scaled_all_sizes< 0.3) * (all_ious<1)].tolist(), (scaled_all_sizes[(scaled_all_sizes< 0.3) * (all_ious<1)]).tolist(), bins=10, colorbar='b',
    colorbar_kw={'length': 1, 'label': "Count"})
axs[0].set_xlabel("Conn. Comp. JI between analysts \n (only when 0 < JI < 1)")
axs[0].set_ylabel(r"Conn. Comp. Volume (ml)")
axs[0].set_title("Analyst JI vs Voxel count histogram per \n Connected Component " + r"(Vol. < 0.3 ml)")

axs[1].scatter((scaled_all_sizes[all_ious<1]).tolist(), (scaled_all_match_sizes[all_ious<1]).tolist(), c=all_ious[all_ious<1].tolist(), alpha=0.5, cmap='summer', colorbar='b',
    colorbar_kw={'length': 1, 'label': "JI"})
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel(r"Analyst 1 Conn. Comp. Volume (ml)")
axs[1].set_ylabel(r"Analyst 2 Conn. Comp. Volume (ml)")
axs[1].set_title("Volume of corresponding Connected \n Components between analysts (0 < JI < 1)")
axs[1].set_xlim((None, 100))
axs[1].set_ylim((None, 100))

axs[2].hist(all_ious.tolist(), bins=20, color='goldenrod')
axs[2].set_xlabel("Conn. Comp. JI between analysts")
axs[2].set_ylabel("Count")
axs[2].set_title("Histogram of Connected Component JI \n between analysts (JI > 0)")

ji0_arr = scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes < 0.3].tolist()
bins = len(ji0_arr) // 30
axs[3].hist(ji0_arr, bins=bins, label='JI = 0', color='#d50032')
ji1_arr = (scaled_all_sizes[(all_ious==1) * (scaled_all_sizes < 0.3)]).tolist()
bins = len(ji1_arr) // 30
if bins > 0:
    axs[3].hist(ji1_arr, bins=bins, label='JI = 1', color='#002c53')
axs[3].legend(loc='b')
axs[3].set_xlabel(r"Conn. Comp. Volume (ml)")
axs[3].set_ylabel("Count")
axs[3].set_title("Histogram of Volume of Conn. Comps. \n with JI of 0 or 1 between analysts")
axs[3].set_xlim((0, 0.3))

axs.format(abc=True, abcloc='ul')
for ax in axs:
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
    ax.spines['left'].set_linewidth(1.2)  # Thicker left spine

fig.save(f"Connected Component information PV REGION {ds_name}.pdf", bbox_inches="tight")

In [ ]:
plt.hist([scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes > 0.3].tolist(), scaled_all_sizes[(all_ious==1) * (scaled_all_sizes > 0.3)].tolist()], color=['#d50032', '#002c53'], bins=20, label=['JI = 0', 'JI = 1'])
plt.legend()
plt.xlabel(r"Conn. Comp. Volume (ml)")
plt.ylabel("Count")
plt.title("Histogram of sizes of large Conn. Comps. \n with JI of 0 or 1 between analysts")

ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
ax.spines['left'].set_linewidth(1.2)  # Thicker left spine
    
plt.savefig(f"Size of large CCS with 0or1 JI PV REGION {ds_name}.pdf", bbox_inches="tight")



## run cc analysis across Deep region

In [ ]:
# for i in range(len(means)):
#     print(xs3d_test[i].shape, pv_region_masks[i].shape)

In [ ]:
all_sizes = []
all_ious = []
all_match_sizes = []
IOU0_ccs_sizes = []

ent_no_overlap_all, ent_low_overlap_all, ent_high_overlap_all, ent_exact_overlap_all = [], [], [], []

none_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
low_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
high_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
exact_prop_uncert_all = [[] for _ in range(len(uncertainty_thresholds))]
non_useful_ccs_all = [[] for _ in range(len(uncertainty_thresholds))]

none_cc_mean = []
low_cc_mean = []
high_cc_mean = []
exact_cc_mean = []

for i in tqdm(range(len(means))):
    x = xs3d_test[i].cuda()
    pred = means[i].cuda().argmax(dim=1)
    e = ent_maps[i].cuda()
    deep_region = 1 - pv_region_masks[i].cuda()
    if x.shape[1] != deep_region.shape[0] or x.shape[2] != deep_region.shape[1] or x.shape[3] != deep_region.shape[2]:
        continue
    y0 = rater0[i].cuda() * deep_region
    y1 = rater1[i].cuda() * deep_region
    
    ccs_y0 = cc3d.connected_components(y0.type(torch.int32).cpu().numpy(), connectivity=26) # 26-connected
    ccs_y0 = torch.from_numpy(ccs_y0.astype(np.float32)).cuda()
    
    ccs_y1 = cc3d.connected_components(y1.type(torch.int32).cpu().numpy(), connectivity=26) # 26-connected
    ccs_y1 = torch.from_numpy(ccs_y1.astype(np.float32)).cuda()
    
    y0 = y0.cuda()
    y1 = y1.cuda()
    
    diff_image = (y0 + y1) == 1
    
    # find type 1: connected components in either map with no overlap
    no_overlap_ccs_y0, no_overlap_ccs_size_y0, no_overlap_ccs_img_y0 = find_non_overlapping_ccs(ccs_y0, y1)
    no_overlap_ccs_y1, no_overlap_ccs_size_y1, no_overlap_ccs_img_y1 = find_non_overlapping_ccs(ccs_y1, y0)
    IOU0_ccs_sizes.extend(no_overlap_ccs_size_y0 + no_overlap_ccs_size_y1)
    combined_no_overlap_image = (no_overlap_ccs_img_y0 + no_overlap_ccs_img_y1).type(torch.int)
    
    # get information on the IOU of connected components between rater 0 and rater 1 where the IOU > 0
    ious, y0_ccids, y1_ccid_matches, sizes, match_sizes = find_ious_between_raters(ccs_y0, ccs_y1, y0, y1)
    all_ious.append(ious)
    all_sizes.append(sizes)
    all_match_sizes.append(match_sizes)
    
    ious = torch.Tensor(ious)
    y0_ccids = torch.Tensor(y0_ccids)
    y1_ccid_matches = torch.Tensor(y1_ccid_matches)
    
    # find type 2: areas where connected components have a poor overlap IOU of 0.5 and below but not 0.
    # IOU of zero ccs have already been discounted in the previous step.
    low_ious = ious < 0.5
    low_IOU_y0_ccs = y0_ccids[low_ious]
    low_IOU_y1_ccs = y1_ccid_matches[low_ious]
    low_IOU_y1_ccs = torch.unique(low_IOU_y1_ccs)
    combined_low_overlap_image = create_overlap_image(low_IOU_y0_ccs, low_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_low_overlap_diff_image = combined_low_overlap_image * diff_image
    
    # find type 3: high overlap
    high_ious = (ious >= 0.5) * (ious < 1)
    high_IOU_y0_ccs = y0_ccids[high_ious]
    high_IOU_y1_ccs = y1_ccid_matches[high_ious]
    high_IOU_y1_ccs = torch.unique(high_IOU_y1_ccs)
    combined_high_overlap_image = create_overlap_image(high_IOU_y0_ccs, high_IOU_y1_ccs, ccs_y0, ccs_y1).type(torch.int)
    combined_high_overlap_diff_image = combined_high_overlap_image * diff_image
    
    # find type 4: areas of exact overlap
    exact_ious = ious == 1
    exact_IOU_y0_ccs = y0_ccids[exact_ious]
    exact_IOU_y1_ccs = y1_ccid_matches[exact_ious]
    exact_IOU_y1_ccs = torch.unique(exact_IOU_y1_ccs)
    combined_exact_overlap_image = create_overlap_image(exact_IOU_y0_ccs, None, ccs_y0, ccs_y1).type(torch.int)
    
    # then get the areas where the raters agree and remove those areas. see what we are left with...
    # that is the joint uncertainty error overlap... but I want to look at the overlap away from the boundaries....
    
    # my concern is that we still have all of this edge information.
    
    # ANALYSIS 1 cc average uncertainty information
    # we need average uncertainty per connected component in the following regions
    # - IR IOU 0
    # - IR IOU 1
    # - diff of IR 0 < IOU < 0.5 regions
    # - diff of IR 0.5 <= IOU < 1 regions
    
    # ANALYSIS 1.5 pixel wise information
    # the same as analysis 1 but with individual voxels
    ef = e.flatten()
    ent_no_overlap, ent_low_overlap, ent_high_overlap, ent_exact_overlap = ef[combined_no_overlap_image.flatten()==1], ef[combined_low_overlap_diff_image.flatten()==1], ef[combined_high_overlap_diff_image.flatten()==1], ef[combined_exact_overlap_image.flatten()==1]
    ent_no_overlap_all.extend(ent_no_overlap.cpu())
    ent_low_overlap_all.extend(ent_low_overlap.cpu())
    ent_high_overlap_all.extend(ent_high_overlap.cpu())
    ent_exact_overlap_all.extend(ent_exact_overlap.cpu())
    
    
    # Analysis 2 connected component extra analysis
    # as the uncertainty threshold increases we look at:
    # average proportion of IR IOU 0 that are uncertain
    # average proportion of IR low IOU that are uncertain
    # average proportion of IR IOU high that are uncertain
    # average proportion of IR IOU 1 that are uncertain
    # number of connected components in uncertainty map that have zero overlap with either rater. I should do this for a fewer number of connected components
    none_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    low_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    high_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    exact_prop_uncert = [[] for _ in range(len(uncertainty_thresholds))]
    non_useful_ccs = [[] for _ in range(len(uncertainty_thresholds))]
    
    ccs_none = torch.from_numpy(cc3d.connected_components(combined_no_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_low = torch.from_numpy(cc3d.connected_components(combined_low_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_high = torch.from_numpy(cc3d.connected_components(combined_high_overlap_diff_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    ccs_exact = torch.from_numpy(cc3d.connected_components(combined_exact_overlap_image.type(torch.int32).cpu().numpy(), connectivity=26).astype(np.float32)).cuda()
    # print(ccs_none.unique())
    # print(combined_no_overlap_image.sum())
    for ti, t in enumerate(uncertainty_thresholds):
        et = (e > t).type(torch.float32)
        
        for cc_id in ccs_none.unique():
            if cc_id == 0:
                continue
            cc = ccs_none == cc_id
            # print(cc_id)
            # print(cc.sum())
            none_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                none_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_low.unique():
            if cc_id == 0:
                continue
            cc = ccs_low == cc_id
            low_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                low_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_high.unique():
            if cc_id == 0:
                continue
            cc = ccs_high == cc_id
            high_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                high_cc_mean.append(e[cc==1].mean())
        for cc_id in ccs_exact.unique():
            if cc_id == 0:
                continue
            cc = ccs_exact == cc_id
            exact_prop_uncert_all[ti].append(et[cc==1].mean().item())
            if ti == 0:
                exact_cc_mean.append(e[cc==1].mean())
                
                
    # Analysis 2.5:
    # average number of superfluous connected components across the dataset as the threshold increases.
            
    
    # Analysis 3 UIRO outside of the regions of the segmentation (do euclidian distance or different components, square and cross for example?) Nice.
    
    # Analysis 4 JUEO outside the regions of the segmentation produced by the model (provide this with 2 different components or do euclidian distance?)
    
    # Analysis 5 pcitures
    # Collect a number of images of different individuals, the different maps, as I have outlined below, and demonstrate the things that we are talking about.
    # Just provide an image every 10 patients or something to demonstrate the examples. 
    # Nice.
    
    # yes this is a really good way to approach the analysis and I actually like it quite a lot.
    # then I need to collect my nice plot of connected components for each dataset and also collect some nice visual examples from the different datasets as well.....
    
    
    # if i == 1:
    #     break
    # break

In [ ]:
islice=30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=7)
b = 20
axs[0].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(y1.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[2].set_title("analyst 2 ")
axs[3].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[3].set_title("ccs with no overlap (in 3D) \n IOU = 0")
axs[4].imshow(combined_low_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[4].set_title("ccs with low overlap (in 3D)\n 0 < IOU < 0.5 ")
axs[5].imshow(combined_high_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[5].set_title("ccs with high overlap (in 3D) \n 0.5 <= IOU < 1")
axs[6].imshow(combined_exact_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[6].set_title("ccs with exact overlap (in 3D)\n IOU = 1")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')
        
fig.savefig("Example of overlap types under consideration DEEP REGION.png", bbox_inches='tight', dpi=600)

In [ ]:
islice = 30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=7)
b = 20
axs[0].imshow(x.cpu()[0, islice][b:-b, b:-b], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.type(torch.float32).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(combined_no_overlap_image.cpu()[islice][b:-b, b:-b], cmap='gray')
axs[2].set_title("ccs with no overlap (in 3D) \n IOU = 0")
axs[3].imshow((diff_image * combined_low_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')
axs[3].set_title("low IOU disagreement region")
axs[4].imshow((diff_image * combined_high_overlap_image).cpu()[islice][b:-b, b:-b], cmap='gray')

axs[4].set_title("high IOU disagreement region")
axs[5].imshow(ent_maps[i].cpu()[islice][b:-b, b:-b], cmap='Oranges')
axs[5].set_title("uncertainty")
axs[6].imshow(means[i][islice].argmax(dim=0).cpu()[b:-b, b:-b], cmap='Greens')
axs[6].set_title("model prediction")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')
        
# fig.savefig("Example of overlap and uncertainty.png", bbox_inches='tight', dpi=600)

In [ ]:
ds_name

In [ ]:
islice=30
fig = pplt.figure(space=0, refwidth='20em')
axs = fig.subplots(nrows=1, ncols=5)
axs[0].imshow(x.cpu()[0, islice], cmap='gray')
axs[0].set_title("FLAIR")
axs[1].imshow(y0.cpu()[islice], cmap='gray')
axs[1].set_title("analyst 1")
axs[2].imshow(ccs_y0.cpu()[islice], cmap='Set2')
axs[2].set_title("analyst 1 ccs")
axs[3].imshow(y1.cpu()[islice], cmap='gray')
axs[3].set_title("analyst 2 ")
axs[4].imshow(ccs_y1.cpu()[islice], cmap='Set2')
axs[4].set_title("analyst 2 ccs")
axs.format(grid=False, xlocator='null', ylocator='null')
for ax in axs:
    for spine in ax.spines:
        ax.spines[spine].set_color('none')

In [ ]:
all_ious = torch.cat([torch.Tensor(t) for t in all_ious])
all_sizes = torch.cat([torch.Tensor(t) for t in all_sizes])
all_match_sizes = torch.cat([torch.Tensor(t) for t in all_match_sizes])
IOU0_ccs_sizes = torch.Tensor(IOU0_ccs_sizes)

In [ ]:
import seaborn as sns
from matplotlib.ticker import NullLocator

In [ ]:
fig = pplt.figure(space=6, refwidth='20em', share=False)
axs = fig.subplots(nrows=1, ncols=4)

scale = 0.003
scaled_all_sizes = all_sizes * scale
scaled_IOU0_ccs_sizes = IOU0_ccs_sizes * scale
scaled_all_match_sizes = all_match_sizes * scale

axs[0].hist2d(all_ious[(scaled_all_sizes< 0.3) * (all_ious<1)].tolist(), (scaled_all_sizes[(scaled_all_sizes< 0.3) * (all_ious<1)]).tolist(), bins=10, colorbar='b',
    colorbar_kw={'length': 1, 'label': "Count"})
axs[0].set_xlabel("Conn. Comp. JI between analysts \n (only when 0 < JI < 1)")
axs[0].set_ylabel(r"Conn. Comp. Volume (ml)")
axs[0].set_title("Analyst JI vs Voxel count histogram per \n Connected Component " + r"(Vol. < 0.3 ml)")

axs[1].scatter((scaled_all_sizes[all_ious<1]).tolist(), (scaled_all_match_sizes[all_ious<1]).tolist(), c=all_ious[all_ious<1].tolist(), alpha=0.5, cmap='summer', colorbar='b',
    colorbar_kw={'length': 1, 'label': "JI"})
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel(r"Analyst 1 Conn. Comp. Volume (ml)")
axs[1].set_ylabel(r"Analyst 2 Conn. Comp. Volume (ml)")
axs[1].set_title("Volume of corresponding Connected \n Components between analysts (0 < JI < 1)")
axs[1].set_xlim((None, 100))
axs[1].set_ylim((None, 100))

axs[2].hist(all_ious.tolist(), bins=20, color='goldenrod')
axs[2].set_xlabel("Conn. Comp. JI between analysts")
axs[2].set_ylabel("Count")
axs[2].set_title("Histogram of Connected Component JI \n between analysts (JI > 0)")

ji0_arr = scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes < 0.3].tolist()
bins = len(ji0_arr) // 30
axs[3].hist(ji0_arr, bins=bins, label='JI = 0', color='#d50032')
ji1_arr = (scaled_all_sizes[(all_ious==1) * (scaled_all_sizes < 0.3)]).tolist()
bins = len(ji1_arr) // 30
if bins > 0:
    axs[3].hist(ji1_arr, bins=bins, label='JI = 1', color='#002c53')
axs[3].legend(loc='b')
axs[3].set_xlabel(r"Conn. Comp. Volume (ml)")
axs[3].set_ylabel("Count")
axs[3].set_title("Histogram of Volume of Conn. Comps. \n with JI of 0 or 1 between analysts")
axs[3].set_xlim((0, 0.3))

axs.format(abc=True, abcloc='ul')
for ax in axs:
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
    ax.spines['left'].set_linewidth(1.2)  # Thicker left spine

fig.save(f"Connected Component information DEEP REGION {ds_name}.pdf", bbox_inches="tight")

In [ ]:
plt.hist([scaled_IOU0_ccs_sizes[scaled_IOU0_ccs_sizes > 0.3].tolist(), scaled_all_sizes[(all_ious==1) * (scaled_all_sizes > 0.3)].tolist()], color=['#d50032', '#002c53'], bins=20, label=['JI = 0', 'JI = 1'])
plt.legend()
plt.xlabel(r"Conn. Comp. Volume (ml)")
plt.ylabel("Count")
plt.title("Histogram of sizes of large Conn. Comps. \n with JI of 0 or 1 between analysts")

ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_linewidth(1.2)  # Thicker bottom spine
ax.spines['left'].set_linewidth(1.2)  # Thicker left spine
    
plt.savefig(f"Size of large CCS with 0or1 JI DEEP REGION {ds_name}.pdf", bbox_inches="tight")

